In [1]:
import numpy as np
import brevitas.nn as qnn
from brevitas.core.quant import QuantType
from torch.nn import Module
from torch import nn
from torch import Tensor
from torch import from_numpy
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from brevitas.quant_tensor import pack_quant_tensor

In [6]:
class XorDataset(Dataset):
    def __init__(self):
        a = Tensor([0,0])
        b = Tensor([0,1])
        c = Tensor([1,0])
        d = Tensor([1,1])
        aq = pack_quant_tensor(a,4,1)
        bq = pack_quant_tensor(b,4,1)
        cq = pack_quant_tensor(c,4,1)
        dq = pack_quant_tensor(d,4,1)
        self.data=[aq,bq,cq,dq]
        self.key=[0.,1.,1.,0.]

    def __getitem__(self, index):
        x = self.data[index]
        y = self.key[index]
        
        return x, y
    
    def __len__(self):
        return len(self.data)

In [7]:
class QuantXORNet(Module):
    def __init__(self):
        super(QuantXORNet, self).__init__()
        #self.batch1 = qnn.QuantBatchNorm2d(num_features=n_inputs, 
        #                                  momentum=0.6,
        #                                  bias_quant_type=QuantType.INT)
        self.relu0 = qnn.QuantReLU(quant_type=QuantType.INT,
                                  bit_width=4,
                                  max_val=8)
        self.linear1 = qnn.QuantLinear(in_features = 2, 
                                       out_features=2, 
                                       bias_quant_type=QuantType.INT,
                                       bias=True,
                                       compute_output_scale=True,
                                       compute_output_bit_width=True,
                                       #input_bit_width=32,
                                       weight_quant_type=QuantType.INT)
        self.relu1 = qnn.QuantReLU(quant_type=QuantType.INT,
                                  bit_width=4,
                                  max_val=8)
        #self.batch2 = qnn.QuantBatchNorm2d(num_features=31, 
        #                                   momentum=0.6,
        #                                   bias_quant_type=QuantType.FP)
        self.linear2 = qnn.QuantLinear(in_features = 2, 
                                       out_features=2, 
                                       bias_quant_type=QuantType.INT, 
                                       bias=True,
                                       compute_output_scale=True,
                                       compute_output_bit_width=True,
                                       #bit_width=4,
                                       weight_quant_type=QuantType.INT)
        self.relu2 = qnn.QuantReLU(quant_type=QuantType.INT,
                                  bit_width=4,
                                  max_val=8)
        #self.batch3 = qnn.QuantBatchNorm2d(num_features=11, 
        #                                   momentum=0.6,
        #                                   bias_quant_type=QuantType.FP)
        self.linear3 = qnn.QuantLinear(in_features = 2, 
                                       out_features=1, 
                                       bias_quant_type=QuantType.INT, 
                                       bias=True,
                                       compute_output_scale=True,
                                       compute_output_bit_width=True,
                                       #bit_width=4,
                                       weight_quant_type=QuantType.INT)
    def forward(self, x):
        res=x
        #res = self.batch1(res)
        #res = self.relu0(res)
        res = self.linear1(res)
        res = self.relu1(res)
        #res = self.relu1(self.linear1(res))
        #res = self.batch2(res)
        res = self.relu2(self.linear2(res))
        #res = self.batch3(res)
        #res = self.relu3(self.linear3(res))
        #res = self.batch4(res)
        res = self.linear3(res)
        return res

In [8]:
model = QuantXORNet()

In [12]:
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
criterion = nn.MSELoss()
loader = DataLoader(XorDataset())

for epoch in range(2):  # loop over the dataset multiple times
    model.train()
    criterion.train()
    running_loss = 0.0
    for i, data in enumerate(loader):
        # zero the parameter gradients
        optimizer.zero_grad()
        inputs,target = data

        # forward + backward + optimize
        print(inputs)
        outputs = model(inputs)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

QuantTensor(tensor=tensor([[0., 0.]]), scale=tensor([4]), bit_width=tensor([1]))


RuntimeError: 
round_vml_cpu not implemented for 'Long' (operator() at /opt/conda/conda-bld/pytorch_1556653099582/work/build/aten/src/ATen/native/cpu/UnaryOpsKernel.cpp.AVX2.cpp:212)
frame #0: c10::Error::Error(c10::SourceLocation, std::string const&) + 0x45 (0x7fd130cdbdc5 in /opt/conda/lib/python3.6/site-packages/torch/lib/libc10.so)
frame #1: <unknown function> + 0x11b7a31 (0x7fd1320aca31 in /opt/conda/lib/python3.6/site-packages/torch/lib/libcaffe2.so)
frame #2: at::native::_round_out_cpu(at::Tensor&, at::Tensor const&) + 0xcb (0x7fd1316e2c4b in /opt/conda/lib/python3.6/site-packages/torch/lib/libcaffe2.so)
frame #3: <unknown function> + 0x92aa68 (0x7fd13181fa68 in /opt/conda/lib/python3.6/site-packages/torch/lib/libcaffe2.so)
frame #4: at::native::round(at::Tensor const&) + 0x69 (0x7fd1316dffc9 in /opt/conda/lib/python3.6/site-packages/torch/lib/libcaffe2.so)
frame #5: at::TypeDefault::round(at::Tensor const&) const + 0x55 (0x7fd131982955 in /opt/conda/lib/python3.6/site-packages/